In [1]:
!pip install sqlalchemy

In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd


class CustomeDataSet():
    def __init__(self, traindata_url, transform=None, size=(768, 768)):
        # Recuperando dados com dataframe --> Pelo visto é a melhor forma ...
        self.df = pd.read_sql("SELECT * FROM dados_table", con=create_engine(f'sqlite:///{traindata_url}/database.db') )
        self.traindata_url = traindata_url # path da pasta com os dados
        self.nome_perfil = self.df['nome_perfil'] # Serie com os nome dos perfis
        self.solicitacao = self.df['solicitacao'] # Serie com as solicitacoes utilizadas
        self.img_uuid = self.df['uuid'] # Identificador unico da aplicação
        self.size = size # Tamanho padrão da imagem
        self.transform = transform # Transformações a serem aplicadas nas imagens


    def __len__(self):
        return len(self.df)


    def __getitem__(self, index):

        # Recuperando uuid do elemento
        uuid_str = self.img_uuid[index]
        img_uuid = f"{uuid_str[:8]}-{uuid_str[8:12]}-{uuid_str[12:16]}-{uuid_str[16:20]}-{uuid_str[20:]}"

        # Carregando as imagens
        img_root = os.path.join('data', 'img', f'{img_uuid}.png')
        img_mask = os.path.join('data', 'mask', f'{img_uuid}.png')
        img_raw = os.path.join('data', 'raw', f'{img_uuid}.png')

        # Escrevendo o propmt da imagem
        
        caption = f'Complete the masked region on the left with the cross-sectional view and the masked region on the right with the plan view of a steel structural connection between an I-beam {self.nome_perfil[index]} and an H-column, designed for a vertical load of {self.solicitacao[index]} kN. Use technical representation according to structural drawing standards, including visible profiles, bolts, and connection plates. Both views must correspond to the same connection.'

        # Aplicando alguma transformação as imagens
        if self.transform:
            img_root = self.transform.image_processor.preprocess(img_root, height=height, width=width).to(device='cuda', dtype=torch.float16)
            img_mask = self.transform.image_processor.preprocess(img_mask, height=height, width=width).to(device='cuda', dtype=torch.float16)
            img_raw = self.transform.image_processor.preprocess(img_raw, height=height, width=width).to(device='cuda', dtype=torch.float16)

        return caption, img_root, img_mask, img_raw

In [3]:
dados = CustomeDataSet(r'C:\Users\Breno HM Rodrigues\Desktop\Data')

In [4]:

# Escreve o arquivo com separador "#;#"
with open("data/train.csv", "w", encoding="utf-8") as f:
    for i in range(10):
        caption, img_root, img_mask, img_raw = dados[i]
        linha = "#;#".join([
            img_raw.replace('\\', '/'),
            img_mask.replace('\\', '/'),
            '',
            img_root.replace('\\', '/'),
            caption,
            'local_editing'
        ])
        f.write(linha + "\n")

print("Arquivo 'data/train.csv' salvo com sucesso!")

Arquivo 'data/train.csv' salvo com sucesso!
